In [1]:
using BenchmarkTools

In [2]:
function MaxPool_v1(input, pool_size)
    input_height, input_width, channels = size(input)
    pool_height, pool_width = pool_size
    
    output_height = div(input_height, pool_height)
    output_width = div(input_width, pool_width)
    
    output = zeros(Float32, output_height, output_width, channels)
    
    for c in 1:channels
        for i in 1:output_height
            for j in 1:output_width
                row_start = 1 + (i-1)*pool_height
                row_end = row_start + pool_height-1
                col_start = 1 + (j-1)*pool_width
                col_end = col_start + pool_width-1
            
                pool = input[row_start:row_end, col_start:col_end, c]
                output[i, j, c] = maximum(pool)
            end
        end
    end
    return output
end


MaxPool_v1 (generic function with 1 method)

In [3]:
function MaxPool_v2(input, pool_size)
    input_rows, input_columns, channels = size(input)
    pool_height, pool_width = pool_size
    
    output_rows = div(input_rows, pool_height)
    output_columns = div(input_columns, pool_width)
    
    output = zeros(Float32, output_rows, output_columns, channels)
    
    for c in 1:channels
        for col in 1:output_columns
            for row in 1:output_rows
                row_start = 1 + (row-1)*pool_height
                row_end = row_start + pool_height-1
                col_start = 1 + (col-1)*pool_width
                col_end = col_start + pool_width-1
            
                pool = @view input[row_start:row_end, col_start:col_end, c]
                output[row, col, c] = maximum(pool)
            end
        end
    end
    return output
end

MaxPool_v2 (generic function with 1 method)

In [5]:
input = rand(Float32, 28,28,6);

In [6]:
@benchmark MaxPool_v1(input, (2,2))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  40.166 μs …  2.750 ms  ┊ GC (min … max): 0.00% … 97.10%
 Time  (median):     41.667 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   43.912 μs ± 45.268 μs  ┊ GC (mean ± σ):  3.61% ±  4.16%

   ▁▅▂▃█▅▁                                                     
  ▂███████▆▅▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  40.2 μs         Histogram: frequency by time        55.6 μs <

 Memory estimate: 96.75 KiB, allocs estimate: 1177.

In [7]:
@benchmark MaxPool_v2(input, (2,2))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.458 μs … 53.958 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.417 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.053 μs ±  2.030 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▅                █                                      
  █▆▁▂▃█▃▂▂▃▂▂▂▄▂▂▂▂▂▂▂▄█▄▃▃▃▃▂▂▂▁▂▂▂▂▂▂▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  13.5 μs         Histogram: frequency by time        21.6 μs <

 Memory estimate: 4.88 KiB, allocs estimate: 1.

In [9]:
MaxPool_v1(input, (2,2)) == MaxPool_v2(input, (2,2))

true